In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import pickle
import datetime

In [2]:
def name_formatter(name):
    name = re.sub("_", " ", name).strip().title()
    return(name)

need to do pip3 install lxml and then reload the jupyter file

In [3]:
def ar_lookup(name):
    """
    Convert names from English to Arabic using European Media Monitors and 
     code modified from Phil Schrodt.
    """
    name = name_formatter(name)
    base_url = "http://emm.newsexplorer.eu/NewsExplorer/search/en/entities?query="
    url = base_url + name
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "lxml")
        name_url = soup.find("p", {"class" : "center_headline"}).find("a")['href']
    except Exception as e:
        #print("Couldn't get page of results back: ", e)
        return []

    try:
        base = "http://emm.newsexplorer.eu/NewsExplorer/search/en/"
        name_url = base + name_url
        name_page = requests.get(name_url)
        soup = BeautifulSoup(name_page.content, "lxml")
        # check to make sure in list???? Take in alt names???/petrarch2/petrarch2/data/dictionaries
        names = soup.find("td", {"colspan" : "1"}).find_all("p")
        names = [i.text for i in names][1:]
        names_en = [i for i in names if re.search("\(.*?Eu|\(.*?en", i)]
        names_en = [re.sub("\s+?\(.+?\)", "", name) for name in names_en]
        #print("Found match. Matched English name: ", names_en[0])
        names_ar = [i for i in names if re.search("\(.*?ar", i)]
        names_ar = [re.sub("\s+?\(.+?\)", "", name) for name in names_ar]
        return names_ar
    except Exception:
        traceback.print_exc()
        return []

In [4]:
ar_lookup("ANGELA_MERKEL")

['أنجيلا ميركل', 'أنجيلا ميركيل']

In [5]:
def clean_line(line):
    # Take out extra space, underscores, comments, etc.
    cleaned = re.sub("_* .+", "", line).strip()
    cleaned = re.sub("_$", "", cleaned, flags=re.MULTILINE)
    return cleaned

def ingest_dictionary(dict_path):
    """
    Read in the country (or other) actor dictionaries.
    """
    with open(dict_path) as f:
        country_file = f.read()
    split_file = country_file.split("\n")
    
    dict_dict = []
    key_name = ""
    alt_names = [] 
    roles = []

    for line in split_file:
        if not line:
            pass
        elif line[0] == "#":
            pass
        elif re.match("[A-Z]", line[0]):
            # handle the previous
            entry = {"actor_en" : key_name,
                    "alt_names_en" : alt_names,
                    "roles" : roles}
            dict_dict.append(entry)
            # zero everything out
            alt_names = []
            roles = []
            # make new key name
            key_name = clean_line(line)
            # check to see if the role is built in
            if bool(re.search("\[[A-Z]{3}\]", line)):
                roles = re.findall("\[(.+?)\]", line)
        elif line[0] == "+":
            cleaned = clean_line(line[1:])
            alt_names.append(cleaned)
        elif re.match("\s", line):
            roles.append(line.strip())
    return dict_dict 
dp = "./Phoenix.Countries.actors.txt"
dict_dict = ingest_dictionary(dp)

In [6]:
len(dict_dict)

18390

In [14]:
dict_dict[1123]

{'actor_en': 'MANSUR_BIN_RAJAB',
 'alt_names_en': [],
 'roles': ['[BHRGOV 070101-100831]']}

In [33]:
ar_lookup("Mohammad_Najibullah")

[]

In [34]:
#dict_dict here is just one object, not a list of object
def eng_to_ar(dict_dict):
    """
    Update an English language dictionary entry with Arabic names
    """
    ar_names = ar_lookup(dict_dict["actor_en"])
    if not ar_names:
        #print("No ar name match found for "+ str(dict_dict["actor_en"]))
        raise Exception(dict_dict["actor_en"])
        #return dict_dict
    dict_dict['actor_ar'] = ar_names[0]
    if len(ar_names) > 1:
        dict_dict['alt_names_ar'] = ar_names[1:]
    return dict_dict

In [72]:
def grabAllTheEnglishNamesThatNoArName(dict_dict):
    noFindList=[]
    for item in dict_dict:
        if(item['actor_en']!=""):
            try:
                eng_to_ar(item)
            except Exception as e:
                noFindList.append(item['actor_en'])
            #print(e)
    return noFindList     

In [71]:
notfind[3]

'SIBGHATULLAH_MOJADEDI'

In [ ]:
%%time
notfind=grabAllTheEnglishNamesThatNoArName(dict_dict)
#then dump the data to pickle
try:
    with open("noFindWord", 'wb') as f:
            pickle.dump(notfind, f, pickle.HIGHEST_PROTOCOL)
except:
    print("failed to save the result to disk")
    pass


In [46]:
notfind

In [ ]:
def hack_wiki(eng_name):
    base_url="https://en.wikipedia.org/wiki/"+eng_name
    try:
        page=requests.get(base_url)
        soup=BeautifulSoup(page.content,"lxml")
        name=soup.find(id="firstHeading").contents
        ar_url=soup.find("li",{"class":"interwiki-ar"}).find("a")['href']
        #print(ar_url)
        ar_page=requests.get(ar_url)
        ar_soup=BeautifulSoup(ar_page.content,"lxml")
        ar_name=ar_soup.find(id="firstHeading").contents
        print(ar_name)
        #print("name "+name+" url: "+str(ar_url))
    except Exception as e:
        print(e)

In [46]:
def eng_to_ar(dict_dict):
    """Screenshot from 2017-06-29 08-36-04
    Update an English language dictionary entry with Arabic names
    """
    ar_names = ar_lookup(dict_dict["actor_en"])
    if not ar_names:
        print("No ar name match found.")
        return dict_dict
    dict_dict['actor_ar'] = ar_names[0]
    if len(ar_names) > 1:
        dict_dict['alt_names_ar'] = ar_names[1:]
    return dict_dict

In [47]:
test=eng_to_ar(dict_dict[7777])

In [48]:
dic=pickle.load(open('countrycode.pkl', 'rb'))

In [49]:
countrycode=[key for key,value in dic.items()]

#Format stored in our db. { "_id" : ObjectId("5772026ca78a30ce5acbec43"), "sentenceId" : "5771f20f0dfcd69f645102fa", "word" : "President Obama", "countryCode" : "USA", "firstRoleCode" : "GOV", "secondRoleCode" : "LEG", "dateStart" : "Sun Jun 01 2008 00:00:00 GMT-0500 (CDT)", "dateEnd" : "Wed Jun 01 2016 00:00:00 GMT-0500 (CDT)", "confidenceFlag" : true, "userId" : "577201cea78a30ce5acbec41", "userName" : "guest", "taggingTime" : ISODate("2016-06-28T04:51:56.092Z"), "__v" : 0 }

In [50]:
test

{'actor_ar': 'أحمد الجلبي',
 'actor_en': 'AHMAD_CHALABI',
 'alt_names_ar': ['أحمد شلبي', 'أحمد جلبي'],
 'alt_names_en': [],
 'roles': ['[IRQELI 620101-030901]',
  '[IRQGOV 030901-030930]',
  '[IRQGOV 031101-040630]',
  '[IRQGOV 050601-060531]',
  '[IRQELI]']}

In [91]:
haha='[IRQELI 620101-030901]'
hahaha=haha.split(" ")

In [94]:
hahaha[1][0]

'6'

In [52]:
from pymongo import MongoClient
client=MongoClient()
client=MongoClient('mongodb://portland.cs.ou.edu:23755/')
db=client['lexisnexis']
db.authenticate('boomer', 'burritos_for_breakfast')
secondroles=db.secondroles
firstroles=db.agents
secondrolelist=[]
for item in secondroles.find():
    secondrolelist.append(item['id'])
firstrolelist=[]
for item in firstroles.find():
    firstrolelist.append(item['id'])

In [53]:
def transferStringToDateTime(inputtime):
    if(str(inputtime)!=''):
        dd=datetime.datetime.strptime(inputtime,'%y%m%d').date()
            #this is for 62 return 1962 not 2062:
        if  dd.year>2017:
            new=dd.replace(year=dd.year-100)
            return new
        else:
            return dd  
    else:
        return ''

In [54]:
transferStringToDateTime("620101")

datetime.date(1962, 1, 1)

In [55]:
#if it is first role it will return 1, if it is in secondRole list it will reture 2, and it it does not exist in both it return 3
def checkBelongToWhichRole(code):
    if code in firstrolelist:
        return 1;
    elif code in secondrolelist:
        return 2;
    else:
        return 3;

In [56]:
def checkIfCountrycodeExist(code):
    if code in countrycode:
        return 1
    else:
        return 0

[AFGGOVMIL >050101] this format of stuff needs to be handled

In [70]:
strtest="IRQELI 620101-030901"
strtest.split(" ")
len("yan")

3

In [71]:
strtest[1:4]

'RQE'

In [95]:
test

{'actor_ar': 'أحمد الجلبي',
 'actor_en': 'AHMAD_CHALABI',
 'alt_names_ar': ['أحمد شلبي', 'أحمد جلبي'],
 'alt_names_en': [],
 'roles': ['[IRQELI 620101-030901]',
  '[IRQGOV 030901-030930]',
  '[IRQGOV 031101-040630]',
  '[IRQGOV 050601-060531]',
  '[IRQELI]']}

In [114]:
#extract the roles and parse the string of each role to get the role format we like for our fajita website
def extract_actor_roles(dict):
    roles=[]
    #it could be ['actor_ar'] is not written since we did not find the translation
    if 'actor_ar' in dict:
        wordlist=[]
        wordlist.append(dict['actor_ar'])
        if 'alt_names_ar' in dict:
            for name in dict['alt_names_ar']:
                wordlist.append(name)
        for word in wordlist:                    
            for item in dict['roles']:
                temp=[]
                splitresults=item.split(" ")
                #if it has both firstRole and secondRole "[" will be include in teh str
                len1=len(splitresults[0])
                country=''
                if(len1==4):
                    country=splitresults[0][1:4]
                    role1=''
                    role2=''
                elif(len1==7):
                    country=splitresults[0][1:4]
                    role1=splitresults[0][4:7]
                    role2=''
                elif(len1==10):
                    country=splitresults[0][1:4]
                    role1=splitresults[0][4:7]
                    role2=splitresults[0][7:10]
                temp.append(country)
                temp.append(checkIfCountrycodeExist(country))
                temp.append(role1)
                temp.append(checkBelongToWhichRole(role1))
                temp.append(role2)
                temp.append(checkBelongToWhichRole(role2))
                if(len(splitresults)>1):
                    timerange=splitresults[1]
                    if(timerange[0]!=">" and timerange[0]!="<"):
                        temp.append(transferStringToDateTime(timerange[0:6]))
                        temp.append("startdate")
                        temp.append(transferStringToDateTime(timerange[7:13]))
                        temp.append("enddate")
                    elif(timerange[0]==">"):
                        temp.append(transferStringToDateTime(timerange[1:7]))
                        temp.append("startdate")
                        temp.append("")
                        temp.append("enddate")
                    elif(timerange[0]==">"):
                        temp.append("")
                        temp.append("startdate")
                        temp.append(transferStringToDateTime(timerange[1:7]))
                        temp.append("enddate")
                else:
                    temp.append("")
                    temp.append("startdate")
                    temp.append("")
                    temp.append("enddate")
                temp.append(word)
                roles.append(temp)    
    return roles
#ok, the format will be
#["contry",'1',"role1","1","role2","2","starttime","something","endtime","something","word"]
        
    

In [115]:
midresult=extract_actor_roles(test)

In [131]:
#generate a list of json object for each element in the list
def generateFajitaObjectsList(midresult):
    finalResult=[]
    for item in midresult:
        tempjson={}
        #insert in all the deault stuff first
        #should be false, the flag is false means we are confident about the word being tagged in fajita.
        tempjson["confidenceFlag"]=False
        tempjson["sentenceId"]="10000"
        tempjson["userName"]="existEnglishDictionary"
        tempjson["userId"]="existEnglishDictionary"
        tempjson["countryCode"]=item[0]
        tempjson["firstRoleCode"]=""
        tempjson["secondRoleCode"]=""
        tempjson["taggingTime"]= datetime.datetime.now().strftime('%Y-%m-%d')
        tempjson["dateStart"]=str(item[6])
        tempjson["dateEnd"]=str(item[7])
        tempjson["word"]=item[10]
        if(item[3]==1):
            tempjson["firstRoleCode"]=item[2]
        elif(item[3]==2):
            tempjson["secondRoleCode"]=item[2]
        else:
            pass
        if(item[5]==1):
            tempjson["firstRoleCode"]=item[4]
        elif(item[5]==2):
            tempjson["secondRoleCode"]=item[4]
        else:
            pass    
        finalResult.append(tempjson)
    return finalResult

In [132]:
final=generateFajitaObjectsList(midresult)

In [133]:
final

[{'confidenceFlag': False,
  'countryCode': 'IRQ',
  'dateEnd': 'startdate',
  'dateStart': '1962-01-01',
  'firstRoleCode': '',
  'secondRoleCode': 'ELI',
  'sentenceId': '10000',
  'taggingTime': '2017-06-24',
  'userId': 'existEnglishDictionary',
  'userName': 'existEnglishDictionary',
  'word': 'أحمد الجلبي'},
 {'confidenceFlag': False,
  'countryCode': 'IRQ',
  'dateEnd': 'startdate',
  'dateStart': '2003-09-01',
  'firstRoleCode': 'GOV',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-24',
  'userId': 'existEnglishDictionary',
  'userName': 'existEnglishDictionary',
  'word': 'أحمد الجلبي'},
 {'confidenceFlag': False,
  'countryCode': 'IRQ',
  'dateEnd': 'startdate',
  'dateStart': '2003-11-01',
  'firstRoleCode': 'GOV',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-24',
  'userId': 'existEnglishDictionary',
  'userName': 'existEnglishDictionary',
  'word': 'أحمد الجلبي'},
 {'confidenceFlag': False,
  'countryCode': 'IRQ',
 

#now need to insert the value back to the database

In [62]:
for item in final:
     db.sourcedictionaries.insert_one(
 {'confidenceFlag': True,
  'countryCode': 'FG',
  'dateEnd': '',
  'dateStart': '',
  'firstRoleCode': '',
  'secondRoleCode': '',
  'sentenceId': '10000',
  'taggingTime': '2017-06-24',
  'userId': 'existEnglishDictionary',
  'userName': 'existEnglishDictionary',
  'word': 'حامد كرزاي'}
)

In [134]:
#filter out the items that does not have roles defined in there:

def iterativeInsertAll(dict_dict):
    insertRecordCount=0
    insertDistinctPersonCount=0
    for item in dict_dict:
        try:
            #print(insertRecordCount)
            temptest=eng_to_ar(item)
            tempmidresult=extract_actor_roles(temptest)
            finalresult=generateFajitaObjectsList(tempmidresult)
            for insertrocord in finalresult:
                try:
                    db.sourcedictionaries.insert_one(insertrocord)
                    insertRecordCount=insertRecordCount+1
                except:
                    pass
            insertDistinctPersonCount=insertDistinctPersonCount+1
        except Exception as e:
            print(e)
            pass
    return {"recordinsert":insertRecordCount,"distinctPersonCount":insertDistinctPersonCount}
        

[AFGGOVMIL >050101]
fromat like this need to handled,
maybe have two actors and have the time format with > or < there.

In [135]:
%%time
totalinsert=iterativeInsertAll(dict_dict)

No ar name match found.
local variable 'role1' referenced before assignment
name 'traceback' is not defined
No ar name match found.
No ar name match found.
unconverted data remains: 01
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
unconverted data remains: 07
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name

No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar na

No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
name 'traceback' is not defined
No ar name match found.
No ar name match found.


No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.


No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar na

...

No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
local variable 'role1' referenced before assignment
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No a

No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match

name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
time data '190001' does not match format '%y%m%d'
time data '1-4501' does not match format '%y%m%d'
time data '1-5301' does not match format '%y%m%d'
name 'traceback' is not defined
unconverted data remains: 01
unconverted data remains: 01
No ar name match found.
No ar name match found.
time data '1-4101' does not match format '%y%m%d'
No ar name match found.
No ar name match found.
No ar name match found.
No ar name mat

No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar na

No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
name 'traceback' is not defined
No ar name match found.
name 'traceback' is not defined
name 'traceback' is not defined
name 'traceback' is not defined
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
No ar name match found.
name 'traceback' is not defined
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
No ar name match found.
name 'traceback' is not defined
name 'tr

In [136]:
totalinsert["recordinsert"]

5696

In [137]:
totalinsert["distinctPersonCount"]

17567